# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2116760572028,
 'week52high': 151.06,
 'week52low': 59.82,
 'week52highSplitAdjustOnly': 149.11,
 'week52lowSplitAdjustOnly': 59.59,
 'week52change': 1.021207381683669,
 'sharesOutstanding': 17092263741,
 'float': 0,
 'avg10Volume': 122749172,
 'avg30Volume': 132274354,
 'day200MovingAvg': 126.68,
 'day50MovingAvg': 131.7,
 'employees': 150716,
 'ttmEPS': 3.71,
 'ttmDividendRate': 0.8475303821482925,
 'dividendYield': 0.006833999163550869,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-29',
 'nextEarningsDate': '2021-01-16',
 'peRatio': 32.47419303902196,
 'beta': 1.1928282193486355,
 'maxChangePercent': 47.22886412566845,
 'year5ChangePercent': 4.062676565915071,
 'year2ChangePercent': 1.66272364697421,
 'year1ChangePercent': 1.203243713344814,
 'ytdChangePercent': -0.07592170488511064,
 'month6ChangePercent': 0.15499240301813436,
 'month3ChangePercent': -0.06397423966090225,
 'month1ChangePercent': -0.02746804902627958,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

1.203243713344814

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,126.755,0.960284,N/A
1,AAL,23.860,1.221025,N/A
2,AAP,190.600,1.321628,N/A
3,AAPL,122.390,1.249704,N/A
4,ABBV,104.600,0.73078,N/A
...,...,...,...,...
500,YUM,112.550,0.986683,N/A
501,ZBH,157.160,0.949814,N/A
502,ZBRA,462.630,1.783361,N/A
503,ZION,54.988,1.332298,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,80.150,7.375964,N/A
1,275,LB,61.080,5.321707,N/A
2,179,FCX,33.630,5.146239,N/A
3,208,GPS,29.390,3.559999,N/A
4,272,KSS,58.300,3.296814,N/A
5,462,URI,313.830,3.26074,N/A
6,305,MGM,39.760,3.197255,N/A
7,212,HAL,22.520,3.153621,N/A
8,441,TPR,43.690,2.998603,N/A
9,135,DISCA,67.780,2.964879,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
portfolio_size

'10000000'

In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

200000.0
d:\tutorial\algorithmic-trading-python\.virtualenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,80.150,7.375964,2495
1,275,LB,61.080,5.321707,3274
2,179,FCX,33.630,5.146239,5947
3,208,GPS,29.390,3.559999,6805
4,272,KSS,58.300,3.296814,3430
5,462,URI,313.830,3.26074,637
6,305,MGM,39.760,3.197255,5030
7,212,HAL,22.520,3.153621,8880
8,441,TPR,43.690,2.998603,4577
9,135,DISCA,67.780,2.964879,2950


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.510,N/A,0.984048,N/A,0.273631,N/A,0.042235,N/A,-0.003699,N/A,N/A
1,AAL,23.436,N/A,1.185372,N/A,0.885371,N/A,0.40611,N/A,0.081586,N/A,N/A
2,AAP,192.099,N/A,1.331747,N/A,0.238558,N/A,0.133238,N/A,0.085614,N/A,N/A
3,AAPL,126.840,N/A,1.211654,N/A,0.151244,N/A,-0.063199,N/A,-0.026958,N/A,N/A
4,ABBV,105.530,N/A,0.713458,N/A,0.239919,N/A,0.027633,N/A,-0.013911,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.990,N/A,0.952271,N/A,0.218726,N/A,0.016366,N/A,0.0324,N/A,N/A
501,ZBH,163.860,N/A,0.993701,N/A,0.177407,N/A,0.050126,N/A,-0.036589,N/A,N/A
502,ZBRA,466.256,N/A,1.756638,N/A,0.885281,N/A,0.226986,N/A,-0.07144,N/A,N/A
503,ZION,56.599,N/A,1.312602,N/A,0.945405,N/A,0.241317,N/A,-0.01117,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [15]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

hqm_dataframe.fillna(value=0, inplace=True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.510,N/A,0.984048,58.613861,0.273631,51.485149,0.042235,36.039604,-0.003699,35.445545,N/A
1,AAL,23.436,N/A,1.185372,69.900990,0.885371,94.653465,0.406110,97.227723,0.081586,87.722772,N/A
2,AAP,192.099,N/A,1.331747,76.435644,0.238558,44.752475,0.133238,63.960396,0.085614,88.712871,N/A
3,AAPL,126.840,N/A,1.211654,71.089109,0.151244,31.089109,-0.063199,7.722772,-0.026958,22.376238,N/A
4,ABBV,105.530,N/A,0.713458,38.019802,0.239919,44.950495,0.027633,30.099010,-0.013911,28.118812,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.990,N/A,0.952271,55.445545,0.218726,40.990099,0.016366,26.138614,0.032400,60.198020,N/A
501,ZBH,163.860,N/A,0.993701,59.603960,0.177407,35.841584,0.050126,38.019802,-0.036589,18.217822,N/A
502,ZBRA,466.256,N/A,1.756638,89.108911,0.885281,94.455446,0.226986,84.356436,-0.071440,6.930693,N/A
503,ZION,56.599,N/A,1.312602,75.841584,0.945405,95.841584,0.241317,86.138614,-0.011170,30.099010,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [12]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: